In [1]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

In [2]:
# Load the dataset
costs_df = pd.read_csv('/Users/mahinbindra/Downloads/nurse_shift_costs.csv')

In [3]:
# Constants from the problem statement
num_nurses = 26
num_shifts = 14
min_hours = 36
max_hours = 60
min_nurses_per_shift = 6
shift_length = 12

In [4]:
# Create a new model
m = Model('nurse_scheduling')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [5]:
# Decision variables
x = m.addVars(num_nurses, num_shifts, vtype=GRB.BINARY, name='x')
y = m.addVars(num_nurses, num_shifts, vtype=GRB.BINARY, name='y')
z = m.addVars(num_nurses, num_shifts, vtype=GRB.BINARY, name='z')

In [6]:
# Objective: Minimize total cost
obj = quicksum(costs_df.at[i, 'Cost_Weekday'] * (x[i, j] + y[i, j] + z[i, j]) for i in range(num_nurses) for j in range(5)) + \
      quicksum(costs_df.at[i, 'Cost_Weekend'] * (x[i, j] + y[i, j] + z[i, j]) for i in range(num_nurses) for j in range(5, num_shifts)) + \
      quicksum(costs_df.at[i, 'Cost_Overtime'] * (quicksum((x[i, j] + y[i, j] + z[i, j]) * shift_length for j in range(num_shifts)) - min_hours) for i in range(num_nurses))
m.setObjective(obj, GRB.MINIMIZE)

In [7]:
# Constraints
# Each shift must have at least 6 nurses
for j in range(num_shifts):
    m.addConstr(quicksum(x[i, j] + y[i, j] + z[i, j] for i in range(num_nurses)) >= min_nurses_per_shift)

# Each nurse works between 36 and 60 hours
for i in range(num_nurses):
    m.addConstr(quicksum((x[i, j] + y[i, j] + z[i, j]) * shift_length for j in range(num_shifts)) >= min_hours)
    m.addConstr(quicksum((x[i, j] + y[i, j] + z[i, j]) * shift_length for j in range(num_shifts)) <= max_hours)

# Each shift must have at least one SRN
for j in range(num_shifts):
    m.addConstr(quicksum(x[i, j] for i in range(num_nurses)) >= 1)

# Nurses cannot work consecutive shifts
for i in range(num_nurses):
    for j in range(num_shifts - 1):
        m.addConstr(x[i, j] + x[i, j+1] <= 1)
        m.addConstr(y[i, j] + y[i, j+1] <= 1)
        m.addConstr(z[i, j] + z[i, j+1] <= 1)
    # Ensure wrap-around constraint for the last shift of the week
    m.addConstr(x[i, 0] + x[i, num_shifts-1] <= 1)
    m.addConstr(y[i, 0] + y[i, num_shifts-1] <= 1)
    m.addConstr(z[i, 0] + z[i, num_shifts-1] <= 1)


In [8]:
# Solve the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1172 rows, 1092 columns and 5824 nonzeros
Model fingerprint: 0xbce85aad
Variable types: 0 continuous, 1092 integer (1092 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective 51572.000000
Presolve time: 0.01s
Presolved: 1172 rows, 1092 columns, 5824 nonzeros
Variable types: 0 continuous, 1092 integer (1092 binary)
Found heuristic solution: objective 44660.000000

Root relaxation: objective 4.326600e+04, 475 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    43266.0

In [9]:
# Output the solution
for v in m.getVars():
    if v.x > 0:
        print(f"{v.varName}, {v.x}")

x[1,1], 1.0
x[2,1], 1.0
x[3,5], 1.0
x[3,7], 1.0
x[3,9], 1.0
x[3,11], 1.0
x[3,13], 1.0
x[4,8], 1.0
x[5,6], 1.0
x[11,9], 1.0
x[12,5], 1.0
x[12,7], 1.0
x[12,12], 1.0
x[13,11], 1.0
x[14,0], 1.0
x[14,2], 1.0
x[15,0], 1.0
x[15,4], 1.0
x[16,0], 1.0
x[16,3], 1.0
x[17,1], 1.0
x[17,5], 1.0
x[18,1], 1.0
x[19,1], 1.0
x[19,10], 1.0
x[20,8], 1.0
x[21,5], 1.0
x[21,11], 1.0
x[21,13], 1.0
x[22,0], 1.0
x[23,11], 1.0
x[24,1], 1.0
x[25,8], 1.0
y[0,10], 1.0
y[0,12], 1.0
y[1,4], 1.0
y[2,4], 1.0
y[4,7], 1.0
y[5,8], 1.0
y[5,13], 1.0
y[6,0], 1.0
y[6,4], 1.0
y[7,12], 1.0
y[8,10], 1.0
y[9,7], 1.0
y[9,12], 1.0
y[10,5], 1.0
y[10,13], 1.0
y[11,12], 1.0
y[12,12], 1.0
y[13,7], 1.0
y[14,2], 1.0
y[18,4], 1.0
y[19,3], 1.0
y[20,6], 1.0
y[22,3], 1.0
y[23,9], 1.0
y[24,2], 1.0
y[24,4], 1.0
y[25,8], 1.0
y[25,10], 1.0
z[0,13], 1.0
z[1,3], 1.0
z[2,2], 1.0
z[4,10], 1.0
z[6,2], 1.0
z[7,5], 1.0
z[7,11], 1.0
z[8,6], 1.0
z[8,9], 1.0
z[9,9], 1.0
z[10,10], 1.0
z[11,6], 1.0
z[12,7], 1.0
z[13,11], 1.0
z[15,3], 1.0
z[16,0], 1.0
z[17,13]

In [11]:
# Number of decision variables = num_nurses * num_shifts * 3 (for SRN, RN, NIT)
total_decision_variables = num_nurses * num_shifts * 3
total_decision_variables

1092

In [13]:
# This would retrieve the optimal value after optimization, which cannot be done here
optimal_value = m.objVal
optimal_value

43266.0